In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
os.getenv("LANGCHAIN_PROJECT")

'Agentic2.0'

In [3]:
os.getenv("LANGCHAIN_API_KEY")

'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'

In [4]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY") # keep these exact names as they would be looked for in langchain
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## Langsmith Tracking And Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true" # this is for langsmith tracing

In [16]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4.1-nano-2025-04-14") # cost optimized model for real time applications
# llm=ChatOpenAI(model="gpt-4.1-mini-2025-04-14") # for more complex tasks
print(llm)



client=<openai.resources.chat.completions.completions.Completions object at 0x000001E26FCE1350> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E26FCE3100> root_client=<openai.OpenAI object at 0x000001E26F9C2EA0> root_async_client=<openai.AsyncOpenAI object at 0x000001E26FCE10F0> model_name='gpt-4.1-nano-2025-04-14' model_kwargs={} openai_api_key=SecretStr('**********')


In [11]:
result=llm.invoke("What is Agentic AI")
print(result)

content='**Agentic AI** refers to artificial intelligence systems that possess or exhibit *agency*—that is, the ability to operate autonomously, make decisions, set goals, and take actions to achieve those goals in a dynamic environment without constant human intervention.\n\n### Key Characteristics of Agentic AI:\n- **Autonomy:** It can perform tasks and make decisions independently.\n- **Goal-oriented behavior:** It can establish or be assigned objectives and work towards achieving them.\n- **Adaptability:** It can perceive changes in its environment and adjust its actions accordingly.\n- **Proactiveness:** Instead of merely reacting to inputs, agentic AI anticipates needs or plans steps ahead.\n\n### Examples and Context:\n- **Intelligent Agents:** Software programs that act on behalf of a user, such as virtual assistants (e.g., Siri, Alexa) that not only respond but can schedule meetings, send reminders, or reorder supplies based on set preferences.\n- **Robotic Systems:** Robots n

In [12]:
print(result.content)

**Agentic AI** refers to artificial intelligence systems that possess or exhibit *agency*—that is, the ability to operate autonomously, make decisions, set goals, and take actions to achieve those goals in a dynamic environment without constant human intervention.

### Key Characteristics of Agentic AI:
- **Autonomy:** It can perform tasks and make decisions independently.
- **Goal-oriented behavior:** It can establish or be assigned objectives and work towards achieving them.
- **Adaptability:** It can perceive changes in its environment and adjust its actions accordingly.
- **Proactiveness:** Instead of merely reacting to inputs, agentic AI anticipates needs or plans steps ahead.

### Examples and Context:
- **Intelligent Agents:** Software programs that act on behalf of a user, such as virtual assistants (e.g., Siri, Alexa) that not only respond but can schedule meetings, send reminders, or reorder supplies based on set preferences.
- **Robotic Systems:** Robots navigating complex e

In [15]:
from langchain_groq import ChatGroq
model=ChatGroq(model="qwen-qwq-32b") # reasoning model - we see it chain of thought
model.invoke("Hi My name is Krish").content

'\n<think>\nOkay, the user said "Hi My name is Krish." I need to respond appropriately. Let me start by greeting them back. I should use their name to make it personal. Maybe say something like "Hello Krish!" to keep it friendly.\n\nNext, I should offer assistance. They might need help with something specific, so asking how I can assist them would be good. I should make sure to keep the tone welcoming and open. Let me check if there\'s anything else I should consider. Since they introduced themselves, maybe they\'re new here. I should encourage them to ask questions or share what they need help with. Alright, that should cover a helpful and friendly response without being too verbose.\n</think>\n\nHello Krish! Nice to meet you. How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything specific! 😊'

 LangChain internally handles string formatting using the Jinja-like templating system. You’re not building the string yourself — you're passing a template object with variable placeholders. LangChain will later inject the value of "input" when you invoke the chain:

 ("user","{input}") -> This is just a tuple of two strings — plain Python. LangChain stores this structure inside a ChatPromptTemplate.

 LangChain parses your message string and searches for placeholders like {input}, then fills them in with values from the dictionary you pass during .invoke(...).


In [20]:
### Prompt Engineering
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer. Provide me answer based on the question"), # system prompt definition
        ("user","{input}") # user prompt. System prompt is used to define the behavior of the model (provided in the background). User prompt is the input to the model.
    ]
)
prompt



ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

String PromptTemplate :
These prompt templates are used to format a single string, and generally are used for simpler inputs. For example, a common way to construct and use a PromptTemplate is as follows:

From <https://python.langchain.com/docs/concepts/prompt_templates/> 
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt_template.invoke({"topic": "cats"})

ChatPromptTemplate:
These prompt templates are used to format a list of messages. These "templates" consist of a list of templates themselves. For example, a common way to construct and use a ChatPromptTemplate is as follows:

from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

In the above example, this ChatPromptTemplate will construct two messages when called. The first is a system message, that has no variables to format. The second is a HumanMessage, and will be formatted by the topic variable the user passes in.

In [17]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E26FBC3750>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E26FBC3C50>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [21]:
### chaining - the idea of langchain is to chain the components together
chain=prompt|model
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E26FBC3750>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E26FBC3C50>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [22]:
response=chain.invoke({"input":"Can you tell me something about Langsmith"})
print(response.content)

As an AI Engineer, I can definitely tell you about LangSmith!

LangSmith is an open-source platform developed by the amazing team at Weights & Biases. It's designed to make the process of fine-tuning large language models (LLMs) much more accessible and efficient. 

Here are some key things to know about LangSmith:

**What it does:**

* **Streamlines fine-tuning:** LangSmith simplifies the often complex process of fine-tuning LLMs for specific tasks. It provides a user-friendly interface and tools that handle many of the technical details, allowing you to focus on your model's performance and goals.
* **Collaboration & Sharing:** It fosters collaboration by enabling users to share their fine-tuned models, datasets, and training scripts with the community. This open approach accelerates progress and allows everyone to benefit from each other's work.
* **Experiment Tracking & Management:** LangSmith integrates seamlessly with Weights & Biases, providing robust experiment tracking capabil

In [23]:
### OutputParser - display the output in a specific format
from langchain_core.output_parsers import StrOutputParser

output_parser=StrOutputParser()

chain=prompt|model|output_parser

response=chain.invoke({"input":"Can you tell me about Langsmith"})
print(response)

Let's talk about Langsmith!

Langsmith is an open-source framework designed to streamline the process of building and deploying language models (LLMs). It's a powerful tool, particularly beneficial for researchers and developers who want to experiment with and customize LLMs without getting bogged down in complex infrastructure or coding. 

Here's a breakdown of what makes Langsmith stand out:

**Key Features:**

* **Simplified LLM Development:** Langsmith provides a user-friendly interface and pre-built components that make it easier to define, train, and evaluate LLMs. This lowers the barrier to entry for newcomers to the field of LLM development.
* **Model Card Generation:**  It automatically generates detailed model cards, which are essential for documenting and understanding the capabilities, limitations, and potential biases of your LLM. 
* **Experiment Tracking:** Langsmith helps you track different model configurations, training runs, and performance metrics. This enables you t

In [24]:
# there are multiple output parsers depending on your needs
from langchain_core.output_parsers import JsonOutputParser

output_parser=JsonOutputParser()
output_parser.get_format_instructions() 
# 'Return a JSON object.' -> this is nothing more than a string which you could add to your system or input prompts

'Return a JSON object.'

String Output Parser mostly works for chaining purposes. You can add it directly to the chain.

But for other parsers, we need to use ChatPromptTemplate.

In [25]:
### OutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

output_parser=JsonOutputParser()

prompt=PromptTemplate( # Answer the user query - your prompt. format_instruction and query are placeholders for the input variables
    template="Answer the user query \n {format_instruction}\n {query}\n ", # instructions for the output parser + what context you want to add
    input_variables=["query"], # these elements (query and format_instruction) will then be fed into template
    partial_variables={"format_instruction":output_parser.get_format_instructions()},
)

prompt=PromptTemplate(
    template="Answer the user query \n Return a JSON object\n {query}\n ", # -> this is the same as the previous prompt!
    input_variables=["query"],
)

In [26]:
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='Answer the user query \n Return a JSON object\n {query}\n ')

In [27]:
chain=prompt|model|output_parser
response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)



{'name': 'Langsmith', 'description': 'Langsmith is an open-source platform for building and deploying language models. It provides a modular and customizable framework for training, evaluating, and serving language models at scale.', 'features': ['Modular architecture', 'Support for various model architectures', 'Easy integration with popular machine learning libraries', 'Scalable training and deployment', 'Comprehensive evaluation tools', 'Open-source and community-driven'], 'benefits': ['Accelerated development cycle', 'Improved model performance', 'Reduced infrastructure costs', 'Enhanced collaboration', 'Access to cutting-edge research'], 'website': 'https://www.langsmith.ai/'}


In [30]:
### Assisgnment ---Chatprompttemplate

# Alternative way to do the same thing using ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system",f"You are an expert AI Engineer. Provide me answer based on the question. {output_parser.get_format_instructions()}"),
    ("user", "{query}")
])
chain = prompt_template | model | output_parser
response = chain.invoke({"query": "Can you tell me about Langsmith?"})
print(response)


### Same done by Krish Naik

# from langchain_core.prompts import ChatPromptTemplate

# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","You are an expert AI Engineer.Provide the response in json.Provide me answer based on the question"),
#         ("user","{input}")
#     ]
# )

# chain=prompt|model|output_parser
# response=chain.invoke({"input":"Can you tell me about Langsmith?"})
# print(response)

{'name': 'Langsmith', 'description': 'Langsmith is an open-source platform for building and deploying AI assistants.', 'key_features': ['**Modular and Extensible:** Built with composable components, allowing for customization and integration with various tools and services.', '**Fine-Tuning Capabilities:** Enables users to fine-tune pre-trained language models on their own datasets for specific tasks.', '**Data Management:** Provides tools for managing and processing conversational data, including data cleaning and annotation.', '**Deployment Flexibility:** Supports deployment on various platforms, including cloud, on-premise, and edge devices.', '**Community-Driven:**  An open-source project with a growing community of developers and contributors.', '**Focus on Privacy:** Emphasizes data privacy and security through features like on-device processing and differential privacy.'], 'use_cases': ['**Chatbots and Conversational Agents:** Building interactive chatbots for customer service, 

In [33]:
from langchain_core.output_parsers import XMLOutputParser
XML_output_parser=XMLOutputParser()

prompt_template = ChatPromptTemplate([
    ("system",f"You are an expert AI Engineer. Provide me answer based on the question. {XML_output_parser.get_format_instructions()}"),
    ("user", "{query}")
])
chain = prompt_template | model | XML_output_parser
response = chain.invoke({"query": "Can you tell me about Langsmith?"})
print(response)

{'LangsmithResponse': [{'description': 'Langsmith is an open-source platform designed to simplify the process of building and deploying large language models (LLMs). It aims to make LLMs more accessible to a wider range of developers and researchers by providing a user-friendly interface and a range of pre-trained models.'}, {'keyFeatures': [{'feature': [{'name': 'Model Hub'}, {'details': 'Offers a collection of pre-trained LLMs, allowing users to easily integrate powerful AI capabilities into their applications without having to train models from scratch.'}]}, {'feature': [{'name': 'Model Training'}, {'details': 'Provides tools and infrastructure for training custom LLMs, enabling users to fine-tune existing models or build entirely new ones tailored to their specific needs.'}]}, {'feature': [{'name': 'Model Deployment'}, {'details': 'Simplifies the process of deploying trained LLMs as APIs, making it easy to integrate them into applications and workflows.'}]}]}]}


### Assigments: https://python.langchain.com/docs/how_to/#prompt-templates

In [34]:
### OutputParser XML format
from langchain_core.output_parsers import XMLOutputParser
output_parser=XMLOutputParser()
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer.<response><answer>Your answer here</answer></response>.Provide me answer based on the question"),
        ("user","{input}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer.<response><answer>Your answer here</answer></response>.Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [35]:
### OutputParser
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate

output_parser=XMLOutputParser()

prompt=PromptTemplate(
    template="Answer the user query \n {format_instruction}\n {query}\n ",
    input_variables=["query"],
    partial_variables={"format_instruction":output_parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a XML file.\n1. Output should conform to the tags below.\n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema.\n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\nNone\n```'}, template='Answer the user query \n {format_instruction}\n {query}\n ')

In [36]:
chain=prompt|model
response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

content="<language-model>\n  <name>Langsmith</name>\n  <description>Langsmith is an open-weights AI assistant built by Cohere. It's designed to be accessible and customizable, allowing developers to fine-tune it for specific tasks and domains.</description>\n  <capabilities>\n    <capability>Text generation</capability>\n    <capability>Text summarization</capability>\n    <capability>Dialogue generation</capability>\n    <capability>Code generation</capability>\n  </capabilities>\n  <open-weights>True</open-weights>\n  <developer>Cohere</developer>\n</language-model> \n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 195, 'total_tokens': 338, 'completion_time': 0.26, 'prompt_time': 0.009263582, 'queue_time': 0.161846782, 'total_time': 0.269263582}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--245e561b-a422-40e2-adba-139230aeca1d-0' usage_metadata={'input_toke

In [37]:
##output parser
#from langchain_core.output_parsers import XMLOutputParser
from langchain.output_parsers.xml import XMLOutputParser

# XML Output Parser
output_parser = XMLOutputParser()

# Prompt that instructs the model to return XML
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond in this XML format: <response><answer>Your answer here</answer></response>"),
    ("human", "{input}")
])

# Build the chain
chain = prompt | model

# Run the chain
#response = chain.invoke({"input": "What is LangChain?"})

raw_output =chain.invoke({"input": "What is LangChain?"})

# Print result
print(raw_output)


content='<response><answer>LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It provides a collection of tools and components that help developers build chains of LLMs, connect them with other data sources, and manage their interactions. </answer></response> \n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 39, 'total_tokens': 106, 'completion_time': 0.121818182, 'prompt_time': 0.002368796, 'queue_time': 0.159619218, 'total_time': 0.124186978}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--7c27dbd6-22cb-4384-8c33-4f8c57a0a419-0' usage_metadata={'input_tokens': 39, 'output_tokens': 67, 'total_tokens': 106}


In [38]:
## With Pydantic
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

model = ChatOpenAI(temperature=0.7) # by default it is gpt-3.5-turbo


# Define your desired data structure. BaseModel is like a data class. Field is like a field in a data class.
# You can combine output parsers with the fields you want this parser to have using pydantic package
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template -> instructions of the fields LLM should return as an output
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two tired!'}

In [40]:
### Without Pydantic
joke_query = "Tell me a joke ."
model = ChatOpenAI(temperature=0.7, model="gpt-4.1-nano-2025-04-14")

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'joke': 'Why did the scarecrow win an award? Because he was outstanding in his field!'}

In [41]:

actor_query = "Generate the shortened filmography for Tom Hanks."

output = model.invoke(
    f"""{actor_query}
Please enclose the movies in <movie></movie> tags"""
)

print(output.content)

Certainly! Here's a shortened filmography of Tom Hanks with the movies enclosed in <movie></movie> tags:

<movie>Forrest Gump</movie>
<movie>Saving Private Ryan</movie>
<movie>Cast Away</movie>
<movie>Big</movie>
<movie>Philadelphia</movie>


In [ ]:
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


model = ChatOpenAI(temperature=0.5, model="gpt-4.1-nano-2025-04-14")

# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke. "

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup='Why did the scarecrow win an award?', punchline='Because he was outstanding in his field!')

### Assisgment:
Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD (integer). use chat Prompt Template.
